## **昨天又没走完** *（因为摸🐟）*，今天根据PyTorch教程中的60min Blitz走第三遭

### Autograd: Automatic Differentiation

PyTorch中所有神经网络的核心是**`autograd`**包

**`autograd`**包为tensors的所有运算提供自动求导，它是一个运行时*（define-by-run）*框架，也就是说反向传播在代码运行时被定义，因而每一次迭代都会不同

#### Tensor

`torch.Tensor`是该包的核心类，如果设置其属性`.requires_grad`为`True`,则它会开始跟踪*（track）*其上的所有运算。计算结束后可以调用`.backward()`自动计算所有梯度。该tensor的梯度将会累积至`.grad`属性中。***（这里的累积（be accumulated）是什么意思？）答：就这么设置的，反向传播时会首先将梯度缓存置零，防止新旧梯度叠加***

要阻止tensor跟踪历史记录，可调用`.detach()`函数将其从计算历史中分离*（detach）*出来，并且能防止将来的计算被跟踪。***看不懂***

要防止跟踪历史记录（同时利用内存），亦可将代码块打包进`with torch.no_grad():`。这在评估一个模型的时候非常有用，因为我们也许有属性`requires_grad=true`的可训练参数，但我们并不需要梯度信息。***这段看不懂啊啊啊***

在自动梯度计算中另一个重要的类是`Function`。

`Tensor`类和`Function`类互联并构建一个非循环图，该图编码了完整的计算历史。每个tensor有一个属性`.grad_fn`,该属性引用产生该`Tensor`的一个`Function`*（除了用户创建的tensors，他们的`grad_fn`为`None`）*

如果你想计算导数*(derivatives)*，可调用`Tensor`对象的`.backward()`方法。如果`Tensor`是一个标量（如只有一个元素），则不需要为`.backward()`指定参数，但如果他有更多的元素，则需要指定一个梯度参数*(gradient argument)*,该参数是一个匹配shape的tensor

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import print_function
import torch

新建一个tensor并设置`requires_grad=True`以跟踪计算

In [23]:
x=torch.ones(2,2,requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


进行tensor运算

In [24]:
# y=torch.ones_like(x)
# torch.add(x,2,out=y)
# 上面的语法会报错add(): functions with out=... arguments don't support automatic differentiation, but one of the arguments requires grad.
y=torch.add(x,2)
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


`y`为运算结果，因而具有`grad_fn`属性

In [25]:
print(x.grad_fn)
print(y.grad_fn)

None


用`y`进行更多运算

In [26]:
z=y*y*3
out=z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


`.requires_grad_(...)`能够就地*(in-place)*改变现存tensor的`requires_grad`属性值, 未指明的tensor该属性默认为False

In [27]:
a=torch.randn(3,2)
a=((a*3)/(a-1))
c=a+2
print(c.grad_fn)
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b=(a*a).sum()
print(b.grad_fn)

None
False
True


##### Gradient

现在进行反向传播。由于`out`只包含一个标量，所以`out.backward()`等效于`out.backward(torch.tensor(1.))`。

In [28]:
out.backward()

打印梯度*d(out)/dx*

In [29]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
